In [28]:
import torch
from torch import nn
import random
from pprint import pprint

import numpy as np

In [29]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(str(device) + ' will be used')

cuda will be used


In [102]:
def __number_sampler(max_digit=3):
    digits = ['0','1','2','3','4','5','6','7','8','9']
    k = random.randint(1,max_digit)
    string = ''.join(random.sample(digits,k))
    if string[0] == '0':
        return __number_sampler(max_digit=max_digit)
    return string

def __operator_sampler():
    operators = ['+','-','/','*']
    return random.choice(operators)

def get_random_input(n_input=100, max_len_limit=15, max_digit=3):
    ret = []
    for i in range(n_input):
        length = random.randint(1,max_len_limit)
        length = length if length%2==1 else length+1
        #length = max_len_limit # added for simplicity, later delete this line to make problem more complicated
        s = ''
        for j in range(length):
            if j % 2 == 0:
                s += __number_sampler(max_digit=max_digit)
            else:
                s += __operator_sampler()
        while len(s) < max_len_limit: s += ' ' ## for variant input
        ret.append(s)
    return ret

def __one_hot_encoding(alphabet, max_len):
    x = np.zeros((len(alphabet),len(alphabet)),dtype=np.int32)
    dict_ = {}
    for i in range(x.shape[0]):
        x[i,i] = 1
        dict_[alphabet[i]] = x[i]
    return dict_

def create_input_seq(alphabet, max_len, input_sequence):
    alphabet2encod = __one_hot_encoding(alphabet, max_len)
    ret = np.zeros((len(input_sequence),max_len,len(alphabet)))
    for j, inp in enumerate(input_sequence):
        temp = np.zeros((len(inp),len(alphabet)))
        for i in range(temp.shape[0]):
            temp[i,:] = alphabet2encod[inp[i]]
        ret[j,:,:] = temp
    return ret

In [103]:
class RNNModel(nn.Module):
    def __init__(self, input_feature_num, hidden_feature_num=200, n_layers=1, out_dim=1):
        super(RNNModel, self).__init__()
        
        self.input_feature_num = input_feature_num
        self.hidden_feature_num = hidden_feature_num
        self.n_layers = n_layers
        self.out_dim = out_dim
        
        self.rnn = nn.RNN(input_feature_num, hidden_feature_num, n_layers)
        self.fc = nn.Linear(hidden_feature_num, out_dim)
        
    def forward(self, x):
        batch_size = x.size(1)
        hidden = self.init_hidden(batch_size)
        hidden = hidden.to(device)
        out, hidden = self.rnn(x, hidden)
        hidden = hidden.contiguous().view(-1, self.hidden_feature_num)
        out = self.fc(hidden)
        return out
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_feature_num)
        return hidden

In [104]:
max_len = 7 # must be an odd number
n_input = 10000
alphabet = ['0','1','2','3','4','5','6','7','8','9','-','+','/','*',' '] 
target_values = []
input_feature_num = len(alphabet)

input_sequence = get_random_input(n_input=n_input, max_len_limit=max_len, max_digit=1)
one_hot_input = create_input_seq(alphabet, max_len, input_sequence)
input_seq = torch.FloatTensor(one_hot_input).permute(1,0,2)

for i in range(len(input_sequence)):
    target_values.append(eval(input_sequence[i]))

target_values = torch.FloatTensor(target_values).unsqueeze(1)
target_values = target_values.to(device)

In [105]:
model = RNNModel(input_feature_num=input_feature_num)
model.to(device)

RNNModel(
  (rnn): RNN(15, 200)
  (fc): Linear(in_features=200, out_features=1, bias=True)
)

In [106]:
n_epochs = 10000
lr=0.01

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = loss_fn.to(device)

In [107]:
for epoch in range(1,n_epochs+1):
    optimizer.zero_grad()
    input_seq = input_seq.to(device)
    output = model(input_seq)
    loss = loss_fn(output, target_values)
    loss.backward()
    optimizer.step()
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/10000............. Loss: 7918.0400
Epoch: 20/10000............. Loss: 7849.9585
Epoch: 30/10000............. Loss: 7819.2495
Epoch: 40/10000............. Loss: 7837.9014
Epoch: 50/10000............. Loss: 7883.2593
Epoch: 60/10000............. Loss: 7876.6846
Epoch: 70/10000............. Loss: 7876.5054
Epoch: 80/10000............. Loss: 7875.5327
Epoch: 90/10000............. Loss: 7874.1953
Epoch: 100/10000............. Loss: 7863.9058
Epoch: 110/10000............. Loss: 7838.8032
Epoch: 120/10000............. Loss: 7842.2871
Epoch: 130/10000............. Loss: 7774.8462
Epoch: 140/10000............. Loss: 7712.2344
Epoch: 150/10000............. Loss: 7635.6768
Epoch: 160/10000............. Loss: 7545.8760
Epoch: 170/10000............. Loss: 7566.1729
Epoch: 180/10000............. Loss: 7226.6768
Epoch: 190/10000............. Loss: 6975.2959
Epoch: 200/10000............. Loss: 6712.2231
Epoch: 210/10000............. Loss: 6863.0913
Epoch: 220/10000............. Loss: 6267.55

KeyboardInterrupt: 

In [89]:
def predict(model, seq):
    one_hot_input = create_input_seq(alphabet, max_len, [seq])
    input_seq = torch.FloatTensor(one_hot_input).permute(1,0,2)
    input_seq = input_seq.to(device)
    output = model(input_seq)
    return output

In [99]:
x = '5+3*2/1'
y = '5-3+5-2'
z = '2/3/5/7'
r = '1+1+1+1'
a = predict(model, x).data

tensor([[5.4266]], device='cuda:0')